#### Importing the required libraries

In [ ]:
from __future__ import division
import rosbag
import numpy as np
import pandas as pd
from scipy import signal
from scipy.spatial.transform import Rotation as R
from tf.transformations import unit_vector, vector_norm, quaternion_conjugate, quaternion_multiply, euler_matrix, quaternion_matrix
from tf.transformations import quaternion_from_matrix, quaternion_about_axis, euler_from_quaternion

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = 'plotly_dark'

#### Importing the data

In [ ]:
# Load rosbag
bag = rosbag.Bag('/home/user/rosbags/2021-12-03-20-27-23.bag')

# Extract data from bag
# Contains raw lidar data and the position/orientation of car
ang_vel = []
lin_acc = []
quats = []
i = 0
# Synch help
msg_imu_t = []
msg_odom_t = []

for topic, msg, t in bag.read_messages(topics='/zed2i/zed_node/imu/data'):
    msg_imu_t.append(t.to_sec())
    av = msg.angular_velocity
    ang_vel.append([av.x, av.y, av.z])
    la = msg.linear_acceleration
    lin_acc.append((la.x, la.y, la.z))
    q = msg.orientation
    quats.append([q.x, q.y, q.z, q.w])
    i += 1
    
odom = []
for topic, msg, t in bag.read_messages(topics='/eGolf/sensors/odometry'):
    msg_odom_t.append(t.to_sec())
    odom.append((
        msg.handwheel_angle, 
        msg.front_right_wheel_speed, 
        msg.front_left_wheel_speed, 
        msg.rear_right_wheel_speed, 
        msg.rear_left_wheel_speed
    ))
    
# Convert to numpy array
ang_vel = np.asarray(ang_vel)
lin_acc = np.asarray(lin_acc)
quats = np.asarray(quats)
odom = np.asarray(odom)

# Frequency defintion
f_imu = 400 # Hz
f_odom = 100 # Hz

# Correct for different rate (IMU is 400 Hz)
"""Get indices of side_array, which are closest in time to main_array"""
def find_closest(main_array, side_array):
    indices = []
    for i,v in enumerate(main_array):
        indices.append((np.abs(side_array - v)).argmin())
    return indices

indices = find_closest(np.asarray(msg_odom_t), np.asarray(msg_imu_t))
# Remove values, such that imu data is 100 Hz as well
# --> imu and odom have now the same length
ang_vel = ang_vel[indices]
lin_acc = lin_acc[indices]
quats = quats[indices]
msg_imu_t_trim = np.asarray(msg_imu_t)[indices]
f_imu = int(f_imu * float(f_odom)/f_imu)

t = np.linspace(0, lin_acc.shape[0]/f_odom, lin_acc.shape[0])

In [ ]:
class TransformImu(object):
    def __init__(self, lin_acc_msg, ang_vel_msg, odom_msg):
        self.lin_acc_msgs = lin_acc_msg
        self.ang_vel_msgs = ang_vel_msg
        self.odom_msgs = odom_msg
        
    def align_imu(self):
        """Get rotation matrix to align imu frame with car frame"""
        # Quaternion to align z-axes of imu and car
        self.quat1 = self.trafo1(self.lin_acc_msgs[:100])

        # Rotation matrix to transform from imu to car frame
        # Check when car starts acceleration
        acc_start_idx = np.argmax(odom[:,1] != 0)
        tf_imu_car = self.trafo2(self.lin_acc_msgs[acc_start_idx:acc_start_idx+100])
        self.is_calibrated = True
        lin_acc_tf, ang_vel_tf = self.transform_imu(tf_imu_car, self.lin_acc_msgs, self.ang_vel_msgs)
        return lin_acc_tf.T, ang_vel_tf.T

    def trafo1(self, lin_acc):
        """First rotation to align imu measured g-vector with car z-axis (up)"""
        # Take average to reduce influence of noise
        lin_acc_avg = np.mean(lin_acc, axis=0)
        # Magnitude of measured g vector (should be around 9.81)
        self.g_mag = vector_norm(lin_acc_avg)

        # Get quaternion to rotate g_imu onto car z-axis
        quat = self.quat_from_vectors(lin_acc_avg, (0, 0, 1))
        return quat

    def trafo2(self, lin_acc):
        """Second rotation to align imu with car frame"""
        # Take average to reduce influence of noise
        lin_acc_avg = np.mean(lin_acc, axis=0)
        # Get rotation matrix from first alignment
        rot_mat1 = quaternion_matrix(self.quat1)[:3, :3]
        # Apply first rotation (trafo1) for z axis alignment
        lin_acc_rot1 = np.inner(rot_mat1, lin_acc_avg)

        # Ignore gravity (because this is already aligned)
        lin_acc_rot1[2] = 0
        # Get quaternion to rotate, such that forward acc is only measured by x-axis
        quat2 = self.quat_from_vectors(lin_acc_rot1, (1, 0, 0))

        # Apply second rotation to first (always in reverse order) to get final rotation
        quat = quaternion_multiply(quat2, self.quat1)
        # Get euler angles to show the difference between the two frames
        euler_angles = euler_from_quaternion(quat)
        print('Correct angles by (rpy in deg): {:.2f} {:.2f} {:.2f}'.format(
            *[np.rad2deg(x) for x in euler_angles]))

        # Get rotation matrix from quaternion
        rot_mat_imu_car = quaternion_matrix(quat)[:3, :3]
        return rot_mat_imu_car

    @staticmethod
    def quat_from_vectors(vec1, vec2):
        """Quaternion that aligns vec1 to vec2"""
        # Make sure both vectors are unit vectors
        v1_uv, v2_uv = unit_vector(vec1), unit_vector(vec2)
        cross_prod = np.cross(v1_uv, v2_uv)
        dot_prod = np.dot(v1_uv, v2_uv)

        # Rotation axis
        axis = cross_prod / vector_norm(cross_prod)
        # Rotation angle (rad)
        ang = np.arctan2(vector_norm(cross_prod), dot_prod)

        # Quaternion ([x,y,z,w])
        quat = np.append(axis*np.sin(ang/2), np.cos(ang/2))
        return quat

    def transform_imu(self, rot_mat, lin_acc, vel_ang):
        """Transforms imu msgs from imu to car frame"""
        lin_acc_tf = np.inner(rot_mat, lin_acc)
        ang_vel_tf = np.inner(rot_mat, vel_ang)

        return lin_acc_tf, ang_vel_tf
    
# Get and apply transformation from imu to car frame
tf_imu = TransformImu(lin_acc, ang_vel, odom)
lin_acc_tf, ang_vel_tf = tf_imu.align_imu()

In [ ]:
fig = px.line(pd.DataFrame(data=np.hstack((ang_vel_tf, ang_vel)), columns=['x','y','z','x1','y1','z1'], index=t))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', title='Angular velocity')
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['x1','y1','z1'] else ())
fig.show()

fig = px.line(pd.DataFrame(data=np.hstack((lin_acc_tf, lin_acc)), columns=['x','y','z','x1','y1','z1'], index=t))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', title='Linear acceleration')
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['x1','y1','z1'] else ())
fig.show()

In [ ]:
from numpy.linalg import norm
import numbers

class Quaternion:
    """
    A simple class implementing basic quaternion arithmetic.
    """
    def __init__(self, w_or_q, x=None, y=None, z=None):
        """
        Initializes a Quaternion object
        :param w_or_q: A scalar representing the real part of the quaternion, another Quaternion object or a
                    four-element array containing the quaternion values
        :param x: The first imaginary part if w_or_q is a scalar
        :param y: The second imaginary part if w_or_q is a scalar
        :param z: The third imaginary part if w_or_q is a scalar
        """
        self._q = np.array([1, 0, 0, 0])

        if x is not None and y is not None and z is not None:
            w = w_or_q
            q = np.array([w, x, y, z])
        elif isinstance(w_or_q, Quaternion):
            q = np.array(w_or_q.q)
        else:
            q = np.array(w_or_q)
            if len(q) != 4:
                raise ValueError("Expecting a 4-element array or w x y z as parameters")

        self._set_q(q)

    # Quaternion specific interfaces

    def conj(self):
        """
        Returns the conjugate of the quaternion
        :rtype : Quaternion
        :return: the conjugate of the quaternion
        """
        return Quaternion(self._q[0], -self._q[1], -self._q[2], -self._q[3])

    def to_angle_axis(self):
        """
        Returns the quaternion's rotation represented by an Euler angle and axis.
        If the quaternion is the identity quaternion (1, 0, 0, 0), a rotation along the x axis with angle 0 is returned.
        :return: rad, x, y, z
        """
        if self.__dict__[0] == 1 and self.__dict__[1] == 0 and self.__dict__[2] == 0 and self.__dict__[3] == 0:
            return 0, 1, 0, 0
        rad = np.arccos(self.__dict__[0]) * 2
        if abs(imaginary_factor) < 1e-8:
            return 0, 1, 0, 0
        x = self._q[1] / imaginary_factor
        y = self._q[2] / imaginary_factor
        z = self._q[3] / imaginary_factor
        return rad, x, y, z

    @staticmethod
    def from_angle_axis(rad, x, y, z):
        s = np.sin(rad / 2)
        return Quaternion(np.cos(rad / 2), x*s, y*s, z*s)

    def to_euler_angles(self):
        pitch = np.arcsin(2 * self.__dict__[1] * self.__dict__[2] + 2 * self.__dict__[0] * self.__dict__[3])
        if np.abs(self.__dict__[1] * self.__dict__[2] + self.__dict__[3] * self.__dict__[0] - 0.5) < 1e-8:
            roll = 0
            yaw = 2 * np.arctan2(self.__dict__[1], self.__dict__[0])
        elif np.abs(self.__dict__[1] * self.__dict__[2] + self.__dict__[3] * self.__dict__[0] + 0.5) < 1e-8:
            roll = -2 * np.arctan2(self.__dict__[1], self.__dict__[0])
            yaw = 0
        else:
            roll = np.arctan2(2 * self.__dict__[0] * self.__dict__[1] - 2 * self.__dict__[2] * self.__dict__[3], 1 - 2 * self.__dict__[1] ** 2 - 2 * self.__dict__[3] ** 2)
            yaw = np.arctan2(2 * self.__dict__[0] * self.__dict__[2] - 2 * self.__dict__[1] * self.__dict__[3], 1 - 2 * self.__dict__[2] ** 2 - 2 * self.__dict__[3] ** 2)
        return roll, pitch, yaw

    def to_euler123(self):
        roll = np.arctan2(-2 * (self.__dict__[2] * self.__dict__[3] - self.__dict__[0] * self.__dict__[1]), self.__dict__[0] ** 2 - self.__dict__[1] ** 2 - self.__dict__[2] ** 2 + self.__dict__[3] ** 2)
        pitch = np.arcsin(2 * (self.__dict__[1] * self.__dict__[3] + self.__dict__[0] * self.__dict__[1]))
        yaw = np.arctan2(-2 * (self.__dict__[1] * self.__dict__[2] - self.__dict__[0] * self.__dict__[3]), self.__dict__[0] ** 2 + self.__dict__[1] ** 2 - self.__dict__[2] ** 2 - self.__dict__[3] ** 2)
        return roll, pitch, yaw

    def __mul__(self, other):
        """
        multiply the given quaternion with another quaternion or a scalar
        :param other: a Quaternion object or a number
        :return:
        """
        if isinstance(other, Quaternion):
            w = self._q[0]*other._q[0] - self._q[1]*other._q[1] - self._q[2]*other._q[2] - self._q[3]*other._q[3]
            x = self._q[0]*other._q[1] + self._q[1]*other._q[0] + self._q[2]*other._q[3] - self._q[3]*other._q[2]
            y = self._q[0]*other._q[2] - self._q[1]*other._q[3] + self._q[2]*other._q[0] + self._q[3]*other._q[1]
            z = self._q[0]*other._q[3] + self._q[1]*other._q[2] - self._q[2]*other._q[1] + self._q[3]*other._q[0]

            return Quaternion(w, x, y, z)
        elif isinstance(other, numbers.Number):
            q = self._q * other
            return Quaternion(q)

    def __add__(self, other):
        """
        add two quaternions element-wise or add a scalar to each element of the quaternion
        :param other:
        :return:
        """
        if not isinstance(other, Quaternion):
            if len(other) != 4:
                raise TypeError("Quaternions must be added to other quaternions or a 4-element array")
            q = self._q + other
        else:
            q = self._q + other._q
            
        return Quaternion(q)

    # Implementing other interfaces to ease working with the class

    def _set_q(self, q):
        self._q = q

    def _get_q(self):
        return self._q

    q = property(_get_q, _set_q)

    def __getitem__(self, item):
        return self._q[item]

    def __array__(self):
        return self._q


class MadgwickAHRS:
    samplePeriod = 1/256
    quaternion = Quaternion(1, 0, 0, 0)
    beta = 1

    def __init__(self, sampleperiod=None, quaternion=None, beta=None):
        """
        Initialize the class with the given parameters.
        :param sampleperiod: The sample period
        :param quaternion: Initial quaternion
        :param beta: Algorithm gain beta
        :return:
        """
        if sampleperiod is not None:
            self.samplePeriod = sampleperiod
        if quaternion is not None:
            self.quaternion = quaternion
        if beta is not None:
            self.beta = beta

    def update(self, gyroscope, accelerometer, magnetometer):
        """
        Perform one update step with data from a AHRS sensor array
        :param gyroscope: A three-element array containing the gyroscope data in radians per second.
        :param accelerometer: A three-element array containing the accelerometer data. Can be any unit since a normalized value is used.
        :param magnetometer: A three-element array containing the magnetometer data. Can be any unit since a normalized value is used.
        :return:
        """
        q = self.quaternion

        gyroscope = np.array(gyroscope, dtype=float).flatten()
        accelerometer = np.array(accelerometer, dtype=float).flatten()
        magnetometer = np.array(magnetometer, dtype=float).flatten()

        # Normalise accelerometer measurement
        if norm(accelerometer) is 0:
            print("accelerometer is zero")
            return
        accelerometer /= norm(accelerometer)

        # Normalise magnetometer measurement
        if norm(magnetometer) is 0:
            print("magnetometer is zero")
            return
        magnetometer /= norm(magnetometer)

        h = q * (Quaternion(0, magnetometer[0], magnetometer[1], magnetometer[2]) * q.conj())
        b = np.array([0, norm(h[1:3]), 0, h[3]])

        # Gradient descent algorithm corrective step
        f = np.array([
            2*(q[1]*q[3] - q[0]*q[2]) - accelerometer[0],
            2*(q[0]*q[1] + q[2]*q[3]) - accelerometer[1],
            2*(0.5 - q[1]**2 - q[2]**2) - accelerometer[2],
            2*b[1]*(0.5 - q[2]**2 - q[3]**2) + 2*b[3]*(q[1]*q[3] - q[0]*q[2]) - magnetometer[0],
            2*b[1]*(q[1]*q[2] - q[0]*q[3]) + 2*b[3]*(q[0]*q[1] + q[2]*q[3]) - magnetometer[1],
            2*b[1]*(q[0]*q[2] + q[1]*q[3]) + 2*b[3]*(0.5 - q[1]**2 - q[2]**2) - magnetometer[2]
        ])
        j = np.array([
            [-2*q[2],                  2*q[3],                  -2*q[0],                  2*q[1]],
            [2*q[1],                   2*q[0],                  2*q[3],                   2*q[2]],
            [0,                        -4*q[1],                 -4*q[2],                  0],
            [-2*b[3]*q[2],             2*b[3]*q[3],             -4*b[1]*q[2]-2*b[3]*q[0], -4*b[1]*q[3]+2*b[3]*q[1]],
            [-2*b[1]*q[3]+2*b[3]*q[1], 2*b[1]*q[2]+2*b[3]*q[0], 2*b[1]*q[1]+2*b[3]*q[3],  -2*b[1]*q[0]+2*b[3]*q[2]],
            [2*b[1]*q[2],              2*b[1]*q[3]-4*b[3]*q[1], 2*b[1]*q[0]-4*b[3]*q[2],  2*b[1]*q[1]]
        ])
        step = j.T.dot(f)
        step /= norm(step)  # normalise step magnitude
        
                # Compute rate of change of quaternion
        qdot = (q * Quaternion(0, gyroscope[0], gyroscope[1], gyroscope[2])) * 0.5 - self.beta * step.T

        # Integrate to yield quaternion
        q += qdot * self.samplePeriod
        self.quaternion = Quaternion(q / norm(q))  # normalise quaternion

    def update_imu(self, gyroscope, accelerometer):
        """
        Perform one update step with data from a IMU sensor array
        :param gyroscope: A three-element array containing the gyroscope data in radians per second.
        :param accelerometer: A three-element array containing the accelerometer data. Can be any unit since a normalized value is used.
        """
        q = self.quaternion

        gyroscope = np.array(gyroscope, dtype=float).flatten()
        accelerometer = np.array(accelerometer, dtype=float).flatten()

        # Normalise accelerometer measurement
        if norm(accelerometer) is 0:
            warnings.warn("accelerometer is zero")
            return
        accelerometer /= norm(accelerometer)

        # Gradient descent algorithm corrective step
        f = np.array([
            2*(q[1]*q[3] - q[0]*q[2]) - accelerometer[0],
            2*(q[0]*q[1] + q[2]*q[3]) - accelerometer[1],
            2*(0.5 - q[1]**2 - q[2]**2) - accelerometer[2]
        ])
        j = np.array([
            [-2*q[2], 2*q[3], -2*q[0], 2*q[1]],
            [2*q[1], 2*q[0], 2*q[3], 2*q[2]],
            [0, -4*q[1], -4*q[2], 0]
        ])
        step = j.T.dot(f)
        step /= norm(step)  # normalise step magnitude

        # Compute rate of change of quaternion
        qdot = (q * Quaternion(0, gyroscope[0], gyroscope[1], gyroscope[2])) * 0.5 - self.beta * step.T

        # Integrate to yield quaternion
        q += qdot * self.samplePeriod
        self.quaternion = Quaternion(q / norm(q))  # normalise quaternion
        return q

### Gravity method

In [ ]:
def car_vel_acc_from_odom(odom):
    wheelbase = 2.631 # m
    alpha = (odom[:,2] - odom[:,3]) / wheelbase
    yaw = alpha * 1.0 / f_odom
    # 3.6 to convert from km/h to m/s
    vel_x_car = ((odom[:,2] + odom[:,3]) / 2) * np.cos(yaw) / 3.6
    acc_x_car = acc_from_vel(vel_x_car)
    return vel_x_car, acc_x_car

def acc_from_vel(vel):
    acc = []
    for i in range(len(vel)-1):
        a = (vel[i+1] - vel[i]) / (1.0/f_odom)
        acc.append(a)
    acc.append(a)
    # Add points if odometry topic recorded less msgs than imu
    msgs_diff = len(lin_acc) - len(odom)
    while msgs_diff > 0:
        acc.append(a)
        msgs_diff -= 1
    return acc

vel_x_car, acc_x_car = car_vel_acc_from_odom(odom)
px.line(acc_x_car)

In [ ]:
px.line(vel_x_car)

In [ ]:
px.line(np.diff(vel_x_car))

In [ ]:
y = vel_x_car
dy = []
for i,v in enumerate(y):
    if i > 2 and i < len(y)-2:
        dy.append((2 * (y[i+1] - y[i-1]) + y[i+2] - y[i-2]) / (8/f_imu) )
    else:
        dy.append(0)
        
px.line(dy)

### Magdwick filter 
from https://github.com/morgil/madgwick_py

In [ ]:
def madgwick(beta=0.5):
    madgwick = MadgwickAHRS(1/f_imu, beta=beta)
    rpy = []
    for i in range(len(lin_acc_tf)):
        # Apply madgwick algorithm to get quaternion estimation
        q = madgwick.update_imu(ang_vel_tf[i], lin_acc_tf[i])
        # Get euler angles in degree
        rpy.append(np.rad2deg(euler_from_quaternion(q)))
    rpy = np.asarray(rpy)
    # Extract pitch angle
    pitch_madgwick = rpy[:,1]
    return pitch_madgwick

px.line(data_frame=pd.DataFrame(np.stack((madgwick(0.9), madgwick(0.5), madgwick(0.1)), axis=1)))

### Complementary filter

In [ ]:
def comp2(lin_acc, ang_vel, K):
    y_acc = []
    y_ang = [0]
    fusion = []
    for i in range(len(lin_acc)):
        # Estimation accelerometer
        y_acc_k = np.arctan2(lin_acc[i][0], np.sqrt(lin_acc[i][1]**2 + lin_acc[i][2]**2))
        # Estimation angular velocity
        y_ang_k = y_ang[-1] + ang_vel[i]/f_imu
        # Fusion estimate of both
        fus = K*y_ang_k + (1-K)*y_acc_k
        
        y_acc.append(y_acc_k)
        y_ang.append(y_ang_k)
        fusion.append(fus)
    y_ang = y_ang[1:]
    return [y_acc, y_ang, fusion]

def complementary_filter(acc_data, gyr_data, alpha):
    angle = 0
    angle_fused = []
    for i,v in enumerate(acc_data):
        angle = (1-alpha)*(angle + gyr_data[i]*(1/f_imu)) + alpha*acc_data[i]
        angle_fused.append(angle)
    return angle_fused

# av = comp2(lin_acc_tf, -ang_vel_tf[:,1], 0.7)
# av = np.asarray(av).T
# px.line(np.rad2deg(av))
        

In [ ]:
vel_angle = -np.degrees(np.cumsum(ang_vel_tf[:,1]) / f_imu)

px.line(vel_angle)

In [ ]:
from scipy.signal import filtfilt, butter

In [ ]:
a, b = butter(5, 0.01, btype='high')

In [ ]:
a, b = butter(5, 0.01, btype='high')
ang_vel_filt = filtfilt(a, b, ang_vel_tf[:,1])
px.line(ang_vel_filt)

In [ ]:
def car_vel_acc_from_odom(odom):
    wheelbase = 2.631 # m
    alpha = (odom[:,2] - odom[:,3]) / wheelbase
    yaw = alpha * 1.0 / f_odom
    # 3.6 to convert from km/h to m/s
    vel_x_car = ((odom[:,2] + odom[:,3]) / 2) * np.cos(yaw) / 3.6
    acc_x_car = acc_from_vel(vel_x_car)
    return vel_x_car, acc_x_car

def acc_from_vel(vel):
    acc = []
    for i in range(len(vel)-1):
        a = (vel[i+1] - vel[i]) / (1.0/f_odom)
        acc.append(a)
    acc.append(a)
    # Add points if odometry topic recorded less msgs than imu
    msgs_diff = len(lin_acc) - len(odom)
    while msgs_diff > 0:
        acc.append(a)
        msgs_diff -= 1
    return acc

vel_x_car, acc_x_car = car_vel_acc_from_odom(odom)
acc_x_imu = acc_rot2[:,0]

fig = px.line(pd.DataFrame(data=vel_x_car, index=t))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-1}]}$', title='Car velocity', showlegend=False)
fig.show()
fig = px.line(pd.DataFrame(data=acc_x_car, index=t))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', title='Car acceleration', showlegend=False)
fig.show()
# fig = px.line(pd.DataFrame(data=acc_x_imu, index=t))
# fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', title='Imu forward acceleration', showlegend=False)
# fig.show()

In [ ]:
# Savitzky-Golay filter
def savitzky_golay(signal):
    y = []
    max_past = 3
    for i,v in enumerate(signal):
        if i < max_past:
            y.append(v)
        else:
            y_k = 0.7 * v + 0.4 * signal[i-1] + 0.1 * signal[i-2] - 0.2 * signal[i-3]
            y.append(y_k)
    return y

def exponential_filter(x):
    a = 0.95
    y = [0]
    for i,v in enumerate(x):
        y_k = a*y[-1] + (1-a)*v
        y.append(y_k)
    y = y[1:]
    return y

data1 = np.stack((acc_x_imu, moving_average(acc_x_imu, 50), savitzky_golay(acc_x_imu),
                 signal.savgol_filter(acc_x_imu, 51, 2), exponential_filter(acc_x_imu)
                 ), axis=1)
data1.shape
# columns1 = ['Orig', 'sav']
fig = px.line(pd.DataFrame(data=data1))
fig.show()

### Filtering the data
Note: When driving on a flat surface the measurements from IMU and odometry should be about the same, but when driving on a ramp they can and should be different

In the first plot it can be seen, that measurements from IMU and odometry almost match each other as long the car is not driving onto a ramp. Because the angle is depended on the difference between those two measurements, this leads to a better car angle, which can be seen in plot 3 (orange vs purple line). The angle now significantly changes when driving onto a ramp, but sometimes the angle still deviates up to 3 degrees on a flat surface (e.g. at 70s).

For the filtering a moving average filter with a window length of 0.5s was used, because it easy to implement and real time ready. But it leads to a slight delay (half of the window length) and also does not perform as well as e.g. a butterworth filter. Further investigation is neccessary to get a better real time ready filtering method.

In [ ]:
def moving_average(signal, window_size):
    """Moving average filter, acts as lowpass filter
    :param val:         Measured value (scalar)
    :param window_size: Window size, how many past values should be considered
    :return filtered:   Filtered signal
    """
    sum = 0
    values = []
    filtered = []
    for i,v in enumerate(signal):
        values.append(v)
        sum += v
        if len(values) > window_size:
            sum -= values.pop(0)
        filtered.append(float(sum) / len(values))
    return np.array(filtered)

acc_x_imu_filt = moving_average(acc_x_imu, 50)
vel_x_car_filt = moving_average(vel_x_car, 50)
acc_x_car_filt = acc_from_vel(vel_x_car_filt)

# Scipy Butterworth filter (also online possible? (filtfilt is definitely only offline))
fc = 1 # Cutoff frequency in Hz
w = fc / (f_imu/2.0) # Normalize fc (Nyquist)
# num, denom of IIR 4-th order butterworth filter with cutoff of 1 Hz
b, a = signal.butter(4, w, 'low')
acc_x_imu_filtButter = signal.filtfilt(b,a, acc_x_imu)
vel_x_car_filtButter = signal.filtfilt(b,a, vel_x_car)
acc_x_car_filtButter = acc_from_vel(vel_x_car_filtButter)

def move_signal(signal, offset):
    """Moves the time of a signal into the future (offset negative) or past"""
    # How many samples to move
    diff_samples = int(offset * f_imu)
    if diff_samples == 0:
        return signal
    # Padding
    pad = np.zeros(abs(diff_samples))
    if offset > 0:
        # Add padding at beginning and remove last values
        signal_padded = np.concatenate((pad, signal))[:-diff_samples]
    else:
        # Add padding at end and remove first values
        signal_padded = np.concatenate((signal, pad))[abs(diff_samples):]
    return signal_padded

def optimize_signal_offset(start, end, signal1=acc_x_imu_filt, signal2=acc_x_car_filt, angle=False):
    # Convert from [s] to samples
    start *= f_imu
    end *= f_imu
    best = [1,0]
    for i in np.linspace(-5,5,1000):
        sig = move_signal(signal1, i) - signal2
        std_dev = np.std(sig[start:end])
        if std_dev < best[0]:
            best = [std_dev, i]
    print('Delay imu signal by {:.2f}s'.format(best[1]))
    if angle:
        return move_signal(signal1, best[1])
    return move_signal(signal1, best[1]) - signal2

data1 = np.vstack([acc_x_imu_filt, acc_x_car_filt, acc_x_imu_filt-acc_x_car_filt, 
                   optimize_signal_offset(0,10), optimize_signal_offset(4,12)]).T
columns1 = ['Filtered IMU', 'Car acc from filtered vel', 
            'IMU cleaned from car acc', 'IMU cleaned with delay (0-10)', 'IMU cleaned with delay (4-12)']

fig = px.line(pd.DataFrame(data=data1, index=t, columns=columns1))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='$\mathrm{[ms^{-2}]}$', 
                  title='Comparison of both acceleration measurement methods')
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['IMU unfiltered','Car acc unfiltered'] else ())
fig.show()

### Using angular velocity measurements to further improve results
After the transformation from imu to car frame a rotation around the y-axis results in the inclination of the car. Therefore the **measured angular velocity of the y-axis** is important.  
A negative value means the car drives a ramp up (or accelerates fast forward) and a positive value means that the car drives a ramp down (or breaks hard).  
To make the comparison with the previous car angle calculation easier, the sign of the angle from gyro measurement has been inverted.  
The z-axis measurement indicate a change of the yaw angle. It should change by 90 deg when driving around a corner. The x-axis measurement is the roll angle and should be near zero all the time. 

The gryoscope measures the angular velocity. By integrating the velocity, the distance (in this case the angle in rad) over time can be calculated. But the gyroscope measurements has white noise, which leads to a drift when integrating, which has to be taken into account.  
For offline data this can be done, by standing still at the start and the end of the recording and then calculating a straight line from start to finish and substract that line from the angle.

In [ ]:
vel_rot = np.inner(tf_imu_car, ang_vel).T
vel_y = vel_rot[:, 1]

fig = px.line(pd.DataFrame(data=vel_rot, index=t, columns=['x','y','z']))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='[rad/s]', title='Angular velocity in car frame coordinates')
fig.show()

ang_y = -np.degrees(np.cumsum(vel_y) / f_imu)
drift = np.linspace(ang_y[0], ang_y[-1], len(ang_y))
ang_y_no_drift = ang_y - drift

fig = px.line(pd.DataFrame(data=np.vstack([ang_y_no_drift, ang_y, drift]).T, index=t, columns=['Corrected','No correction','Drift']))
fig.update_layout(xaxis_title='Time [s]', yaxis_title='[deg]', title='Y-Angle over time')
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['No correction','Drift'] else ())
fig.show()

It can be seen, that the angle drifts about 16 deg per minute without any correction. But even after substracting the estimated drift, the angle still seems to drift slightly. 

#### Complementary filter (similar results to Kalman filter but less complex)
Complementary filter uses the good short term accuracy of the gyroscope and combines it with the accelerometer data, which is good in the long run but not in the short term, to reduce the drifting.  
Hence the name complementary filter. The formula is as follows
$$ \theta = (1-\alpha)(\theta + gyrData\cdot dt) + \alpha(accData)$$
with:
- $\theta$: Estimated angle from fusing measurements
- $\alpha$: Time constant response time in the range of [0-1]. With a value of 0 only the gyroscope is being used and with $\alpha=1$ only the accelerometer.
- $gyrData$: Angular velocity 
- $accData$: Angle from the low-pass filtered accelerometer data using $accData = \arctan2(a_x, a_z)$
    - (in our case $\arctan2(a_x-a_\mathrm{car}, a_z)$ to be precise)
    
A high value of $\alpha$ means that the measurements of the gyroscope are trusted less than the from the accelerometer, and vice versa a small $\alpha$ value means that the are gyroscope measurements are weighted more.  
A decrease of $\alpha$ leads to a smoother signal, but at the cost of adding some addtional time delay.

In [ ]:
def complementary_filter(acc_data, gyr_data, alpha):
    angle = 0
    angle_fused = []
    for i,v in enumerate(acc_data):
        angle = (1-alpha)*(angle + gyr_data[i]*(1/f_imu)) + alpha*acc_data[i]
        angle_fused.append(angle)
    return angle_fused

car_ang = pitch_car_odom(acc_x_imu_filt, acc_x_car_filt)
car_ang2 = pitch_car_odom(optimize_signal_offset(0,10,angle=True), acc_x_car_filt)
    
data1 = np.vstack([ang_y_no_drift, car_ang, car_ang2, pitch_car_odom(acc_x_imu_filtButter, acc_x_car_filtButter),
                   complementary_filter(car_ang, vel_y, 0.01), complementary_filter(car_ang, vel_y, 0.025), 
                   complementary_filter(car_ang2, vel_y, 0.025)]).T
columns1 = ['Gyro w/o drift only', 'IMU + Odom + Moving average', 'IMU + Odom + Moving average delayed', 
            'IMU + Odom + Butterworth', 'Complementary filter - 0.01', 'Complementary filter - 0.025', 'Complf2']
fig = px.line(pd.DataFrame(data=data1, index=t, columns=columns1))
fig.for_each_trace(lambda trace: trace.update(visible='legendonly')
                  if trace.name in ['Gyro w/o drift only', 'IMU + Odom + Butterworth', 'Complementary filter - 0.01'] else ())
fig.update_layout(xaxis_title='Time [s]', yaxis_title='[deg]', title='Y-Angle over time')

##### Testing different parameter values

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=car_ang))
# Add traces, one for each slider step
for step in np.linspace(0.1, 0, 51):
    fig.add_trace(
        go.Scatter(
            visible=False,
            x=t,
            y=complementary_filter(car_ang, vel_y, step)))
fig.data[0].visible = True
# Create and add slider
steps = []
for i in range(len(fig.data)):
    step = dict(
        method="update",
        args=[{"visible": [True] + [False] * len(fig.data)},
              {"title": "Angle using complementary filter with alpha: {:.3f}".format(0.102-i*0.002)}],  # layout attribute
    )
    step["args"][0]["visible"][i] = True  # Toggle i'th trace to "visible"
    steps.append(step)
sliders = [dict(
    active=0,
    currentvalue={"prefix": "alpha: "},
    steps=steps)]
fig.update_layout(sliders=sliders, xaxis_title='Time [s]', yaxis_title='[deg]', title='Complementary Filter compared to IMU+Odom filt with moving avg')
fig.show()